In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.preprocessing import image                  
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.applications.vgg16 import VGG16
from keras import optimizers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir /content/data
!cp -r '/content/drive/My Drive/Colab Notebooks/test_spectograms.zip' /content/data

In [4]:
!unzip /content/data/test_spectograms.zip -d /content/data
!rm -r /content/data/__MACOSX
!rm /content/data/test_spectograms/.DS_Store

Archive:  /content/data/test_spectograms.zip
   creating: /content/data/test_spectograms/
  inflating: /content/data/test_spectograms/20037.png  
  inflating: /content/data/test_spectograms/27758.png  
  inflating: /content/data/test_spectograms/17843.png  
  inflating: /content/data/test_spectograms/26446.png  
  inflating: /content/data/test_spectograms/16585.png  
  inflating: /content/data/test_spectograms/21329.png  
  inflating: /content/data/test_spectograms/27980.png  
  inflating: /content/data/test_spectograms/22620.png  
  inflating: /content/data/test_spectograms/24251.png  
  inflating: /content/data/test_spectograms/8820.png  
  inflating: /content/data/test_spectograms/14392.png  
  inflating: /content/data/test_spectograms/9280.png  
  inflating: /content/data/test_spectograms/23258.png  
  inflating: /content/data/test_spectograms/24537.png  
  inflating: /content/data/test_spectograms/15932.png  
  inflating: /content/data/test_spectograms/12085.png  
  inflating: /co

In [0]:
def load_dataset(path):
    data = load_files(path)
    spectogram_files = np.array(data['filenames'])

    filenames = [filepath.split('/')[4] for filepath in data['filenames']]
    sample_ids = [int(filename.split('.')[0]) for filename in filenames]
    subm_df = pd.DataFrame(sample_ids, columns=['signal_id'])
    subm_df['target'] = 0
    
    return spectogram_files, subm_df

In [0]:
test_files, submission_df = load_dataset('/content/data/')

In [0]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [8]:
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 20337/20337 [00:39<00:00, 520.38it/s]


In [0]:
def cnn_model():
    # Define the network
    image_input = Input(shape=(224, 224, 3))
    pretrained_model = VGG16(weights='imagenet', include_top=False, input_tensor=image_input)

    last_layer = pretrained_model.get_layer('block5_pool').output
    x= Flatten(name='flatten')(last_layer)
    x = Dense(128, activation='relu', name='fc1')(x)
    x = Dense(128, activation='relu', name='fc2')(x)
    out = Dense(2, activation='softmax', name='output')(x)
    custom_model = Model(image_input, out)
    
    for layer in custom_model.layers[:-3]:
        layer.trainable = False
    
    custom_model.compile(optimizer=optimizers.SGD(lr=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return custom_model

In [12]:
model = cnn_model()
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [0]:
!cp '/content/drive/My Drive/Colab Notebooks/saved_weights_transfer_learning.hdf5' /content/
model.load_weights('/content/saved_weights_transfer_learning.hdf5')

In [0]:
test_pred_real = model.predict(test_tensors)
test_pred = np_utils.to_categorical([np.argmax(prediction) for prediction in test_pred_real])

In [15]:
submission_df['target'] = test_pred.argmax(axis=1)
submission_df.head()

,signal_id,target
0,23313,0
1,16398,0
2,21139,0
3,19462,0
4,24499,0


In [0]:
submission_df = submission_df.sort_values(by=['signal_id'])
submission_df.to_csv('/content/kaggle_submission_transfer_learning', index=False)

In [0]:
!cp '/content/kaggle_submission_transfer_learning' '/content/drive/My Drive/Colab Notebooks/kaggle_submission_transfer_learning.csv'